# 1 этап - строю предсказательную модель

In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv('/Users/mamolchanova/Documents/Обучение/ML/19 занятие - гибридные рекомендательные системы/ml-latest/movies.csv')

In [3]:
movies['genres_split'] = movies.genres.str.split('|')

In [4]:
movies['genres_cnt'] = movies.apply(lambda r: len(r['genres_split']), axis=1)

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [6]:
movies['genres_space'] = movies.apply(lambda r: ' '.join(r['genres_split']), axis=1)

In [7]:
cnt_vec_gen = CountVectorizer()

In [8]:
processed_gen = cnt_vec_gen.fit_transform(movies['genres_space'])

In [9]:
tfidf_gen = TfidfTransformer()

In [10]:
tfidf_dense_gen = tfidf_gen.fit_transform(processed_gen).todense()

In [11]:
columns_gen = [None for i in range(len(cnt_vec_gen.vocabulary_))]
for k in cnt_vec_gen.vocabulary_:
    columns_gen[cnt_vec_gen.vocabulary_[k]] = k

In [12]:
df_tfidf_gen = pd.DataFrame(tfidf_dense_gen, columns=columns_gen)

In [13]:
movies_with_tfidf = pd.concat((movies, df_tfidf_gen), axis=1)

In [14]:
import re
movies_with_tfidf['year'] = movies_with_tfidf.apply(lambda r: int(re.findall(r'\(([0-9]{4})\)', r['title'])[0]) if len(re.findall(r'\(([0-9]{4})\)', r['title'])) else None, axis=1)

In [15]:
movies_with_tfidf['year'] = movies_with_tfidf['year'].fillna(movies_with_tfidf['year'].median())

In [16]:
fc_gen = ['movieId', 'title', 'genres_cnt', 'action', 'adventure', 'animation', 'children',
       'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'fi', 'film',
        'horror', 'imax', 'listed', 'musical', 'mystery', 'no',
       'noir', 'romance', 'sci', 'thriller', 'war', 'western', 'year']

In [17]:
movies_for_ds = movies_with_tfidf[fc_gen]

In [18]:
ratings = pd.read_csv('/Users/mamolchanova/Documents/Обучение/ML/19 занятие - гибридные рекомендательные системы/ml-latest/ratings.csv')

In [19]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [20]:
rating_movies = ratings.groupby("movieId")
rating_movies_mean = rating_movies.agg({"rating": "mean"}).rename({'rating': 'rating_movies_mean'}, axis=1)
rating_movies_mean.head()

,rating_movies_mean
movieId,
1,3.886649
2,3.246583
3,3.173981
4,2.874540
5,3.077291


In [21]:
rating_movies_std = rating_movies.agg({"rating": "std"}).rename({'rating': 'rating_movies_std'}, axis=1)

In [22]:
rating_movies_mean = rating_movies_mean.reset_index()

In [23]:
rating_movies_std = rating_movies_std.reset_index()

In [24]:
rating_users = ratings.groupby("userId")
rating_users_mean = rating_users.agg({"rating": "mean"}).rename({'rating': 'rating_users_mean'}, axis=1).reset_index()
rating_users_std = rating_users.agg({"rating": "std"}).rename({'rating': 'rating_users_std'}, axis=1).reset_index()

In [25]:
movies_final = pd.merge(movies_with_tfidf, rating_movies_mean, on='movieId')
movies_final = pd.merge(movies_final, rating_movies_std, on='movieId')

In [26]:
raiting_counts = ratings['userId'].value_counts().to_dict()

Т.к. датасет тяжелый, выберу пользователя, для которого буду делать предсказание, до конечной склейки. Выбираю так, чтобы это был не выброс, но с достаточным количеством оценок. 

In [27]:
# for k, v in raiting_counts.items():
#    if (v >= 100)&(v<=110):
#        print(k, ' ', v)

39014   110
203129   110
103155   110
193308   110
232089   110
54733   110
96304   110
57605   110
216588   110
133553   110
74488   110
74366   110
187425   110
170383   110
145770   110
45903   110
249631   110
16046   110
268133   110
93692   110
42797   110
70589   110
120749   110
84147   110
73879   110
11870   110
213743   110
255327   110
143488   110
172578   110
259993   110
190824   110
100383   110
28846   110
15749   110
5528   110
63923   110
138187   110
236602   110
222197   110
702   110
58440   110
211958   110
205825   110
262769   110
226734   110
97438   110
27890   110
118737   110
227169   110
160759   110
8682   110
100369   110
198782   110
102634   110
98128   110
275324   110
217558   110
245117   110
88259   110
95300   110
242364   110
270660   110
247497   110
116911   110
225426   110
143554   110
127065   110
143295   110
246915   110
10628   110
114134   110
276315   110
77675   110
138088   110
137409   110
153122   110
230861   110
238629   110
80431

In [28]:
TARGET_USER = 213743
ratings_user = ratings[ratings['userId']==TARGET_USER]
df_for_user = pd.merge(movies_final, ratings_user, on='movieId')

In [29]:
df_for_user.head()

,movieId,title,genres,genres_split,genres_cnt,genres_space,action,adventure,animation,children,...,sci,thriller,war,western,year,rating_movies_mean,rating_movies_std,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fantasy]",5,Adventure Animation Children Comedy Fantasy,0.00000,0.442224,0.493386,0.489546,...,0.0,0.0,0.000000,0.0,1995.0,3.886649,0.929551,213743,3.5,1499192133
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[Adventure, Children, Fantasy]",3,Adventure Children Fantasy,0.00000,0.536347,0.000000,0.593741,...,0.0,0.0,0.000000,0.0,1995.0,3.246583,0.959502,213743,3.5,1499192162
2,104,Happy Gilmore (1996),Comedy,[Comedy],1,Comedy,0.00000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,1996.0,3.394278,1.076980,213743,2.5,1499191972
3,110,Braveheart (1995),Action|Drama|War,"[Action, Drama, War]",3,Action Drama War,0.53892,0.000000,0.000000,0.000000,...,0.0,0.0,0.776407,0.0,1995.0,4.008481,0.971303,213743,3.0,1495285525
4,318,"Shawshank Redemption, The (1994)",Crime|Drama,"[Crime, Drama]",2,Crime Drama,0.00000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,1994.0,4.424188,0.763178,213743,4.5,1494182197


In [30]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

In [31]:
df_for_user.shape

(110, 36)

In [32]:
for i in df_for_user.columns:
    df_for_user[i]=df_for_user[i].fillna(0)

In [33]:
y = df_for_user['rating']

In [34]:
X = df_for_user.drop(['rating', 'userId', 'movieId', 'timestamp', 'title', 'genres', 'genres_split', 'genres_space'], axis=1)

In [35]:
model = RandomForestRegressor(max_depth=4, random_state=0, n_estimators=100)

In [36]:
model.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [37]:
print("r2_train: {:.4f}, mae_train: {:.4f}".format(
        model.score(X ,y),
        mean_absolute_error(model.predict(X), y)
    ))

r2_train: 0.6878, mae_train: 0.3661


In [38]:
movies_final = pd.merge(movies_final, ratings_user, how='left', on='movieId')

In [39]:
movies_final['userId'] = movies_final.userId.replace(0, TARGET_USER)

In [40]:
for i in movies_final.columns:
    movies_final[i]=movies_final[i].fillna(0)

In [41]:
X_for_pred = movies_final.drop(['rating', 'userId', 'movieId', 'timestamp', 'title', 'genres', 'genres_split', 'genres_space'], axis=1)

In [43]:
movies_final['y_pred'] = model.predict(X_for_pred)

In [44]:
movies_final = movies_final.sort_values(by=['y_pred'], ascending=False).head(100)

Если взглянуть на рекомендации, то уже сейчас они выглядят достаточно интересно и в топе находятся неплохие фильмы. А у многих фильмов можно найти общие признаки.

In [45]:
movies_final

,movieId,title,genres,genres_split,genres_cnt,genres_space,action,adventure,animation,children,...,thriller,war,western,year,rating_movies_mean,rating_movies_std,userId,rating,timestamp,y_pred
10615,40815,Harry Potter and the Goblet of Fire (2005),Adventure|Fantasy|Thriller|IMAX,"[Adventure, Fantasy, Thriller, IMAX]",4,Adventure Fantasy Thriller IMAX,0.000000,0.400385,0.000000,0.000000,...,0.323454,0.0,0.0,2005.0,3.768579,0.978550,213743.0,5.0,1.494182e+09,4.298355
16249,81834,Harry Potter and the Deathly Hallows: Part 1 (...,Action|Adventure|Fantasy|IMAX,"[Action, Adventure, Fantasy, IMAX]",4,Action Adventure Fantasy IMAX,0.337237,0.398344,0.000000,0.000000,...,0.000000,0.0,0.0,2010.0,3.879580,1.003638,213743.0,4.5,1.495285e+09,4.254463
11996,54001,Harry Potter and the Order of the Phoenix (2007),Adventure|Drama|Fantasy|IMAX,"[Adventure, Drama, Fantasy, IMAX]",4,Adventure Drama Fantasy IMAX,0.000000,0.413492,0.000000,0.000000,...,0.000000,0.0,0.0,2007.0,3.774758,1.003861,213743.0,4.5,1.495285e+09,4.245210
4898,4993,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy,"[Adventure, Fantasy]",2,Adventure Fantasy,0.000000,0.666554,0.000000,0.000000,...,0.000000,0.0,0.0,2001.0,4.097943,1.005020,213743.0,4.5,1.495285e+09,4.190564
5854,5952,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy,"[Adventure, Fantasy]",2,Adventure Fantasy,0.000000,0.666554,0.000000,0.000000,...,0.000000,0.0,0.0,2002.0,4.074705,0.995181,213743.0,4.5,1.495285e+09,4.186397
7769,8368,Harry Potter and the Prisoner of Azkaban (2004),Adventure|Fantasy|IMAX,"[Adventure, Fantasy, IMAX]",3,Adventure Fantasy IMAX,0.000000,0.423131,0.000000,0.000000,...,0.000000,0.0,0.0,2004.0,3.802977,0.958801,213743.0,4.5,1.495285e+09,4.185513
7042,7153,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy,"[Action, Adventure, Drama, Fantasy]",4,Action Adventure Drama Fantasy,0.470991,0.556334,0.000000,0.000000,...,0.000000,0.0,0.0,2003.0,4.102853,1.003048,213743.0,4.5,1.495285e+09,4.172007
13973,69844,Harry Potter and the Half-Blood Prince (2009),Adventure|Fantasy|Mystery|Romance|IMAX,"[Adventure, Fantasy, Mystery, Romance, IMAX]",5,Adventure Fantasy Mystery Romance IMAX,0.000000,0.365040,0.000000,0.000000,...,0.000000,0.0,0.0,2009.0,3.848672,0.999531,213743.0,4.5,1.495285e+09,4.163240
17571,88125,Harry Potter and the Deathly Hallows: Part 2 (...,Action|Adventure|Drama|Fantasy|Mystery|IMAX,"[Action, Adventure, Drama, Fantasy, Mystery, I...",6,Action Adventure Drama Fantasy Mystery IMAX,0.303408,0.358385,0.000000,0.000000,...,0.000000,0.0,0.0,2011.0,3.911401,1.010871,213743.0,4.5,1.495285e+09,4.130973
5718,5816,Harry Potter and the Chamber of Secrets (2002),Adventure|Fantasy,"[Adventure, Fantasy]",2,Adventure Fantasy,0.000000,0.666554,0.000000,0.000000,...,0.000000,0.0,0.0,2002.0,3.635519,0.991433,213743.0,4.5,1.495286e+09,4.103963


# 2 этап - ранжирование с помощью библиотеки surprise

In [46]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

In [47]:
from surprise.model_selection import KFold

In [48]:
kfold = KFold(5)

In [49]:
df_for_surpise = ratings[['userId', 'movieId', 'rating']]

In [50]:
df_for_surpise.columns = ['uid', 'iid', 'rating']

In [52]:
reader = Reader(rating_scale=(0.5, 5))

In [53]:
dataset = Dataset.load_from_df(df_for_surpise, reader)

In [54]:
dataset, _ = train_test_split(dataset, test_size=0.01)

Т.к. датасет тяжелый, выбрала относительно быстрый в обучении алгоритм.

In [55]:
from surprise import BaselineOnly

In [56]:
algo = BaselineOnly()

In [57]:
algo.fit(dataset)

Estimating biases using als...


In [85]:
movies_final

,movieId,title,genres,genres_split,genres_cnt,genres_space,action,adventure,animation,children,...,thriller,war,western,year,rating_movies_mean,rating_movies_std,userId,rating,timestamp,y_pred
17571,88125,Harry Potter and the Deathly Hallows: Part 2 (...,Action|Adventure|Drama|Fantasy|Mystery|IMAX,"[Action, Adventure, Drama, Fantasy, Mystery, I...",6,Action Adventure Drama Fantasy Mystery IMAX,0.303408,0.358385,0.000000,0.000000,...,0.000000,0.0,0.0,2011.0,3.911401,1.010871,213743.0,4.5,1.495285e+09,4.130973
16249,81834,Harry Potter and the Deathly Hallows: Part 1 (...,Action|Adventure|Fantasy|IMAX,"[Action, Adventure, Fantasy, IMAX]",4,Action Adventure Fantasy IMAX,0.337237,0.398344,0.000000,0.000000,...,0.000000,0.0,0.0,2010.0,3.879580,1.003638,213743.0,4.5,1.495285e+09,4.254463
13973,69844,Harry Potter and the Half-Blood Prince (2009),Adventure|Fantasy|Mystery|Romance|IMAX,"[Adventure, Fantasy, Mystery, Romance, IMAX]",5,Adventure Fantasy Mystery Romance IMAX,0.000000,0.365040,0.000000,0.000000,...,0.000000,0.0,0.0,2009.0,3.848672,0.999531,213743.0,4.5,1.495285e+09,4.163240
11996,54001,Harry Potter and the Order of the Phoenix (2007),Adventure|Drama|Fantasy|IMAX,"[Adventure, Drama, Fantasy, IMAX]",4,Adventure Drama Fantasy IMAX,0.000000,0.413492,0.000000,0.000000,...,0.000000,0.0,0.0,2007.0,3.774758,1.003861,213743.0,4.5,1.495285e+09,4.245210
10615,40815,Harry Potter and the Goblet of Fire (2005),Adventure|Fantasy|Thriller|IMAX,"[Adventure, Fantasy, Thriller, IMAX]",4,Adventure Fantasy Thriller IMAX,0.000000,0.400385,0.000000,0.000000,...,0.323454,0.0,0.0,2005.0,3.768579,0.978550,213743.0,5.0,1.494182e+09,4.298355
7769,8368,Harry Potter and the Prisoner of Azkaban (2004),Adventure|Fantasy|IMAX,"[Adventure, Fantasy, IMAX]",3,Adventure Fantasy IMAX,0.000000,0.423131,0.000000,0.000000,...,0.000000,0.0,0.0,2004.0,3.802977,0.958801,213743.0,4.5,1.495285e+09,4.185513
7042,7153,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy,"[Action, Adventure, Drama, Fantasy]",4,Action Adventure Drama Fantasy,0.470991,0.556334,0.000000,0.000000,...,0.000000,0.0,0.0,2003.0,4.102853,1.003048,213743.0,4.5,1.495285e+09,4.172007
5854,5952,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy,"[Adventure, Fantasy]",2,Adventure Fantasy,0.000000,0.666554,0.000000,0.000000,...,0.000000,0.0,0.0,2002.0,4.074705,0.995181,213743.0,4.5,1.495285e+09,4.186397
5718,5816,Harry Potter and the Chamber of Secrets (2002),Adventure|Fantasy,"[Adventure, Fantasy]",2,Adventure Fantasy,0.000000,0.666554,0.000000,0.000000,...,0.000000,0.0,0.0,2002.0,3.635519,0.991433,213743.0,4.5,1.495286e+09,4.103963
4898,4993,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy,"[Adventure, Fantasy]",2,Adventure Fantasy,0.000000,0.666554,0.000000,0.000000,...,0.000000,0.0,0.0,2001.0,4.097943,1.005020,213743.0,4.5,1.495285e+09,4.190564


In [109]:
scoreSer = []

In [110]:
for i in range(0,65):
    movieId = movies_final.iloc[i]['movieId']
    scoreSer.append(algo.predict(dataset.to_inner_uid(TARGET_USER), dataset.to_inner_iid(movieId)).est)

In [120]:
movies_final = movies_final.head(65)

In [115]:
scoreSer = pd.Series(scoreSer)

In [121]:
movies_final = movies_final.assign(score=scoreSer.values)

# Топ рекомендаций

In [126]:
movies_final.sort_values(by=['score'], ascending=False).head(10)

,movieId,title,genres,genres_split,genres_cnt,genres_space,action,adventure,animation,children,...,war,western,year,rating_movies_mean,rating_movies_std,userId,rating,timestamp,y_pred,score
5854,5952,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy,"[Adventure, Fantasy]",2,Adventure Fantasy,0.0,0.666554,0.0,0.000000,...,0.0,0.0,2002.0,4.074705,0.995181,213743.0,4.5,1.495285e+09,4.186397,3.919322
7769,8368,Harry Potter and the Prisoner of Azkaban (2004),Adventure|Fantasy|IMAX,"[Adventure, Fantasy, IMAX]",3,Adventure Fantasy IMAX,0.0,0.423131,0.0,0.000000,...,0.0,0.0,2004.0,3.802977,0.958801,213743.0,4.5,1.495285e+09,4.185513,3.904306
10615,40815,Harry Potter and the Goblet of Fire (2005),Adventure|Fantasy|Thriller|IMAX,"[Adventure, Fantasy, Thriller, IMAX]",4,Adventure Fantasy Thriller IMAX,0.0,0.400385,0.0,0.000000,...,0.0,0.0,2005.0,3.768579,0.978550,213743.0,5.0,1.494182e+09,4.298355,3.887133
5718,5816,Harry Potter and the Chamber of Secrets (2002),Adventure|Fantasy,"[Adventure, Fantasy]",2,Adventure Fantasy,0.0,0.666554,0.0,0.000000,...,0.0,0.0,2002.0,3.635519,0.991433,213743.0,4.5,1.495286e+09,4.103963,3.823586
11996,54001,Harry Potter and the Order of the Phoenix (2007),Adventure|Drama|Fantasy|IMAX,"[Adventure, Drama, Fantasy, IMAX]",4,Adventure Drama Fantasy IMAX,0.0,0.413492,0.0,0.000000,...,0.0,0.0,2007.0,3.774758,1.003861,213743.0,4.5,1.495285e+09,4.245210,3.782629
42497,166231,Die Salzprinzessin (2015),Children|Fantasy,"[Children, Fantasy]",2,Children Fantasy,0.0,0.000000,0.0,0.703487,...,0.0,0.0,2015.0,5.000000,0.000000,0.0,0.0,0.000000e+00,4.063309,3.609667
48233,179639,Die Bremer Stadtmusikanten (2009),Children|Fantasy,"[Children, Fantasy]",2,Children Fantasy,0.0,0.000000,0.0,0.703487,...,0.0,0.0,2009.0,4.500000,0.000000,0.0,0.0,0.000000e+00,4.063309,3.589423
48186,179545,Der Froschkönig (2008),Children|Fantasy,"[Children, Fantasy]",2,Children Fantasy,0.0,0.000000,0.0,0.703487,...,0.0,0.0,2008.0,4.500000,0.000000,0.0,0.0,0.000000e+00,4.063309,3.589423
45833,173853,The Portuguese Nun (2009),Drama,[Drama],1,Drama,0.0,0.000000,0.0,0.000000,...,0.0,0.0,2009.0,4.250000,0.353553,0.0,0.0,0.000000e+00,4.044179,3.589423
47257,177209,Acı Aşk (2009),Drama,[Drama],1,Drama,0.0,0.000000,0.0,0.000000,...,0.0,0.0,2009.0,5.000000,0.000000,0.0,0.0,0.000000e+00,4.044179,3.589423


# Топ фильмов, которые пользователь еще не видел

In [127]:
movies_final[movies_final['rating']==0].sort_values(by=['score'], ascending=False).head(10)

,movieId,title,genres,genres_split,genres_cnt,genres_space,action,adventure,animation,children,...,war,western,year,rating_movies_mean,rating_movies_std,userId,rating,timestamp,y_pred,score
42497,166231,Die Salzprinzessin (2015),Children|Fantasy,"[Children, Fantasy]",2,Children Fantasy,0.0,0.00000,0.0,0.703487,...,0.0,0.0,2015.0,5.00,0.000000,0.0,0.0,0.0,4.063309,3.609667
53574,193129,Edge of the Garden (2011),Drama|Fantasy,"[Drama, Fantasy]",2,Drama Fantasy,0.0,0.00000,0.0,0.000000,...,0.0,0.0,2011.0,5.00,0.000000,0.0,0.0,0.0,4.051411,3.589423
48222,179617,Dornröschen (2009),Children|Fantasy,"[Children, Fantasy]",2,Children Fantasy,0.0,0.00000,0.0,0.703487,...,0.0,0.0,2009.0,5.00,0.000000,0.0,0.0,0.0,4.063309,3.589423
48189,179551,Der Prinz im Bärenfell (2015),Children|Fantasy,"[Children, Fantasy]",2,Children Fantasy,0.0,0.00000,0.0,0.703487,...,0.0,0.0,2015.0,4.50,0.000000,0.0,0.0,0.0,4.063309,3.589423
48187,179547,Die Gänsemagd (2009),Children|Fantasy,"[Children, Fantasy]",2,Children Fantasy,0.0,0.00000,0.0,0.703487,...,0.0,0.0,2009.0,5.00,0.000000,0.0,0.0,0.0,4.063309,3.589423
48186,179545,Der Froschkönig (2008),Children|Fantasy,"[Children, Fantasy]",2,Children Fantasy,0.0,0.00000,0.0,0.703487,...,0.0,0.0,2008.0,4.50,0.000000,0.0,0.0,0.0,4.063309,3.589423
48170,179513,Lucky's Treasure (2017),Adventure|Children,"[Adventure, Children]",2,Adventure Children,0.0,0.67033,0.0,0.742063,...,0.0,0.0,2017.0,4.50,0.000000,0.0,0.0,0.0,4.052206,3.589423
47571,177957,The Exchange (2011),Drama,[Drama],1,Drama,0.0,0.00000,0.0,0.000000,...,0.0,0.0,2011.0,4.50,0.000000,0.0,0.0,0.0,4.044179,3.589423
47257,177209,Acı Aşk (2009),Drama,[Drama],1,Drama,0.0,0.00000,0.0,0.000000,...,0.0,0.0,2009.0,5.00,0.000000,0.0,0.0,0.0,4.044179,3.589423
45833,173853,The Portuguese Nun (2009),Drama,[Drama],1,Drama,0.0,0.00000,0.0,0.000000,...,0.0,0.0,2009.0,4.25,0.353553,0.0,0.0,0.0,4.044179,3.589423
